In [72]:
from sklearn.model_selection import (RandomizedSearchCV, 
    KFold, cross_val_score, GridSearchCV, train_test_split, StratifiedShuffleSplit)
from sklearn.tree import DecisionTreeClassifier
from scipy. stats import randint
from sklearn.ensemble import RandomForestClassifier

import pandas as pd
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt

In [19]:
# Carregamento dos dados
uri = "https://gist.githubusercontent.com/guilhermesilveira/e99a526b2e7ccc6c3b70f53db43a87d2/raw/1605fc74aa778066bf2e6695e24d53cf65f2f447/machine-learning-carros-simulacao.csv"
dados = pd.read_csv(uri).drop(columns=["Unnamed: 0"], axis=1)
display(dados.head())

x = dados.drop(columns=['vendido'])
y = dados.vendido

,preco,vendido,idade_do_modelo,km_por_ano
0,30941.02,1,18,35085.22134
1,40557.96,1,20,12622.05362
2,89627.50,0,12,11440.79806
3,95276.14,0,3,43167.32682
4,117384.68,1,4,12770.11290


# 01. Randomized Search e Cross Validation combinadas

Com a mesma ideia do GridSearchCV de procurar os hiperparâmetros ótimos existe o RandomizedSearchCV, sendo que a diferença é que o GridSearch procura por todo o espaço de parâmetros fornecido e o RandomizedSearch calcula apenas um número de valores aleatórios, especificado pelo parâmetro n_iter da função.

In [23]:
np.random.seed(333)

espaco_de_parametros = {
    "max_depth" : list(range(2, 11)),
    "min_samples_split": np.multiply(list(range(1, 11)), dados.shape[0]/100).astype(int),
    "min_samples_leaf": np.multiply(list(range(1, 11)), dados.shape[0]/100).astype(int),
    "criterion": ["gini", "entropy"]
}

# Instanciando e ajustando 
busca = RandomizedSearchCV(DecisionTreeClassifier(), espaco_de_parametros, n_iter= 16,
                           cv=KFold(n_splits=5, shuffle=True))
busca.fit(x, y)
    
# Armazenando resultados num DataFrame
resultados = pd.DataFrame(busca.cv_results_)
resultados.head()

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_min_samples_split,param_min_samples_leaf,param_max_depth,param_criterion,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.008378,0.001354,0.002192,0.000399,400,800,2,entropy,"{'min_samples_split': 400, 'min_samples_leaf':...",0.7780,0.7500,0.7480,0.7585,0.7545,0.7578,0.010736,14
1,0.010782,0.000740,0.001795,0.000391,400,500,7,entropy,"{'min_samples_split': 400, 'min_samples_leaf':...",0.7960,0.7675,0.7625,0.7720,0.7705,0.7737,0.011613,5
2,0.008103,0.000679,0.001388,0.000499,100,1000,6,entropy,"{'min_samples_split': 100, 'min_samples_leaf':...",0.7855,0.7475,0.7535,0.7690,0.7465,0.7604,0.014908,11
3,0.007773,0.000968,0.001797,0.000728,900,1000,6,entropy,"{'min_samples_split': 900, 'min_samples_leaf':...",0.7855,0.7475,0.7535,0.7690,0.7465,0.7604,0.014908,11
4,0.006464,0.000601,0.001589,0.000484,700,500,2,gini,"{'min_samples_split': 700, 'min_samples_leaf':...",0.7780,0.7500,0.7480,0.7585,0.7545,0.7578,0.010736,14


In [24]:
np.random.seed(333)
# Score por nested cross validation
scores = cross_val_score(busca, x, y, cv=KFold(n_splits=5, shuffle=True))
print(f'A acurácia obtida está em [{round((scores.mean()-2*scores.std())*100, 2)}%, {round((scores.mean()+ 2*scores.std())*100, 2)}%].')

# Melhor estimador encontrado
print(f'Características do melhor estimador: {busca.best_estimator_}.')

A acurácia obtida está em [77.17%, 80.05%].
Características do melhor estimador: DecisionTreeClassifier(max_depth=3, min_samples_leaf=200, min_samples_split=700).


# 02. Explorando por mais tempo espaços maiores

Já que o espaço de parâmetros é explorado de de maneira aleatória e o tempo total comparado ao GridSearch + validação diminuiu consideravelmente, é possível dar uma abertura maior para os valores de cada parâmetro.

In [42]:
np.random.seed(333)

espaco_de_parametros = {
    "max_depth" : list(range(2, 20)),
    "min_samples_split": randint(0.1*dados.shape[0]/(1e2), 5*dados.shape[0]/(1e2)), #  0.1% a 5% da len
    "min_samples_leaf": randint(0.1*dados.shape[0]/(1e2), 5*dados.shape[0]/(1e2)),
    "criterion": ["gini", "entropy"]
}

# Instanciando e ajustando 
busca = RandomizedSearchCV(DecisionTreeClassifier(), espaco_de_parametros, n_iter=128,
                           cv=KFold(n_splits=5, shuffle=True))
busca.fit(x, y)

# Armazenando resultados num DataFrame
resultados = pd.DataFrame(busca.cv_results_)

In [43]:
np.random.seed(333)
# Score por nested cross validation
scores = cross_val_score(busca, x, y, cv=KFold(n_splits=5, shuffle=True))
print(f'A acurácia obtida está em [{round((scores.mean()-2*scores.std())*100, 2)}%, {round((scores.mean()+ 2*scores.std())*100, 2)}%].')

# Melhor estimador encontrado
print(f'Características do melhor estimador: {busca.best_estimator_}.')

A acurácia obtida está em [77.13%, 80.25%].
Características do melhor estimador: DecisionTreeClassifier(max_depth=4, min_samples_leaf=172, min_samples_split=398).


# 03. Baseline com busca exaustiva no espaço discretizado

Para fazer uma comparação com a busca aleatória, faz-se de uma inspeção pelo GridSearchCV num espaço discretizado medindo o tempo. Ambas as abordagem possuem suas vantages e em um caso real é provável que não sejam testadas as duas já que o tempo gasto seria muito elevado. Apesar disso, é visível que o GridSearch se torna praticamente inviável para espaços de parâmetros com um alta dimensão e muitas valores possíveis. Aqui, são somente 2x2x3x3x2x2 = 144 combinações em 6 dimensões e o tempo total entre GridSearch e cross_val_score foi cerca de 16 minutos.

In [58]:
%%time
np.random.seed(333)

espaco_de_parametros = {
    'n_estimators': [10, 100],
    "max_depth" : [3, 5],
    "min_samples_split": [32, 64, 128], #  0.1% a 5% da len
    "min_samples_leaf": [32, 64, 128],
    'bootstrap': [True, False],
    "criterion": ["gini", "entropy"]
}

busca = GridSearchCV(RandomForestClassifier(), espaco_de_parametros, cv=KFold(n_splits = 5, shuffle=True))
busca.fit(x, y)

resultados = pd.DataFrame(busca.cv_results_)

Wall time: 3min 3s


In [59]:
%%time
np.random.seed(333)

scores = cross_val_score(busca, x, y, cv=KFold(n_splits=5, shuffle=True))
print(f'A acurácia obtida está em [{round((scores.mean()-2*scores.std())*100, 2)}%, {round((scores.mean()+ 2*scores.std())*100, 2)}%].')

# Melhor estimador encontrado
print(f'Características do melhor estimador: {busca.best_estimator_}.')

A acurácia obtida está em [76.15%, 79.21%].
Características do melhor estimador: RandomForestClassifier(max_depth=5, min_samples_leaf=64, min_samples_split=64,
                       n_estimators=10).
Wall time: 12min 52s


# 04. Comparando com busca aleatória

Agora, rodando o RandomizedSearchCV para o mesmo espaço de parâmetros e um n_iter menor que as 144 combinações vistas antes no GridSearchCV.

In [61]:
%%time
np.random.seed(333)

espaco_de_parametros = {
    'n_estimators': [10, 100],
    "max_depth" : [3, 5],
    "min_samples_split": [32, 64, 128], #  0.1% a 5% da len
    "min_samples_leaf": [32, 64, 128],
    'bootstrap': [True, False],
    "criterion": ["gini", "entropy"]
}

busca = RandomizedSearchCV(RandomForestClassifier(), espaco_de_parametros,
                           cv=KFold(n_splits = 5, shuffle=True), n_iter=20)
busca.fit(x, y)

resultados = pd.DataFrame(busca.cv_results_)

Wall time: 27.5 s


In [62]:
%%time
np.random.seed(333)

scores = cross_val_score(busca, x, y, cv=KFold(n_splits=5, shuffle=True))
print(f'A acurácia obtida está em [{round((scores.mean()-2*scores.std())*100, 2)}%, {round((scores.mean()+ 2*scores.std())*100, 2)}%].')

# Melhor estimador encontrado
print(f'Características do melhor estimador: {busca.best_estimator_}.')

A acurácia obtida está em [75.7%, 78.7%].
Características do melhor estimador: RandomForestClassifier(max_depth=5, min_samples_leaf=32, min_samples_split=32,
                       n_estimators=10).
Wall time: 1min 52s


In [63]:
scores.std()

0.0074766302570074845

O resultado foi encontrado em um espaço de tempo bem reduzido: um pouco menos de 2 minutos e meio. Quanto ao resultado em si, de um modo geral, a média foi pior com um desvio menor e o estimador final teve alguns parâmetros diferentes comparado ao encontrado pela varredura completa. Avaliar a validade de paremetrizar pelo GridSearch depende da capacidade computacional e da aplicação (precisão/acurácia desejada), já que, no fim, o resultado não foi tão distante.

# 05. Otimização de hiperparâmetros sem validação cruzada (treino, teste e validação)

Quando não se usa validação cruzada, é necessário deixar uma parte do conjunto de dados separada de antemão para executar a técnica do holdout ao avaliar o modelo; cria-se um conjunto de validação para avaliar o modelo escolhido após o refinamento do período de escolha.

In [69]:
# 60% treino, 20% teste, 20% validação; *em relação a x*
np.random.seed(333)

# Aqui, o parâmetro test_size é usado para separar o conjunto de validação
x_treino_teste, x_validacao, y_treino_teste, y_validacao = train_test_split(x, y, test_size=0.2, 
                                                                            shuffle=True, stratify=y)

In [78]:
%%time
np.random.seed(333)

espaco_de_parametros = {
    "n_estimators" : randint(10, 101),
    "max_depth" : randint(3, 6),
    "min_samples_split": randint(32, 129),
    "min_samples_leaf": randint(32, 129),
    "bootstrap" : [True, False],
    "criterion": ["gini", "entropy"]
}

# Como os valores eram em relação a (x) e, aqui, test_size é informado em relação ao fit feito em
# *x_treino_teste*, ele deve ser convertido para %(x_treino_teste) --> (0.16/0.80 = 0.25)
split = StratifiedShuffleSplit(n_splits=1, test_size=0.25)
busca = RandomizedSearchCV(RandomForestClassifier(), espaco_de_parametros,
                           cv=split, n_iter=20)
busca.fit(x_treino_teste, y_treino_teste)

resultados = pd.DataFrame(busca.cv_results_)

Wall time: 4.13 s


In [79]:
%%time
np.random.seed(333)

scores = cross_val_score(busca, x_validacao, y_validacao, cv=split)
print(f'A acurácia obtida foi de {round(scores[0]*100, 2)}%.')

# Melhor estimador encontrado
print(f'Características do melhor estimador: {busca.best_estimator_}.')

A acurácia obtida foi de 78.0%.
Características do melhor estimador: RandomForestClassifier(max_depth=4, min_samples_leaf=77, min_samples_split=108,
                       n_estimators=11).
Wall time: 1.44 s
